# Pulling subject/genre information

### Book data from Goodreads

In [42]:
import json
import pandas as pd
import numpy as np
import re

In [4]:
## Load Goodreads data
dir = '../data/'
filename = 'book_data_full_isbn.csv'
gr = pd.read_csv(dir + filename)

Some isbn13 numbers are recorded as ASIN (Amazon Standard Identification Number) values. For now, these are being removed in the cleaning step, but it may be a good idea to figure out how to work around this.

### Clean data

#### Cleaning ISBNs

Some ISBN numbers have 'X' as the 13th digit. Apparently this is supposed to belong to only ISBN 10 numbers. It's possible that Open Library has somehow 'constructed' their own ISBN 13 identifiers from existing ISBN 10 numbers when the ISBN 13 identifiers were missing.
Whether or not these should be included will depend on what ISBN values are in the Good Reads data. Currently, any ISBN numbers with alpha digits have been removed from the Goodreads data however. So far, none of them are 13 digit ISBNs that end in X.

In [5]:
## ISBN Numbers

## Change non-valid isbn numbers to "None"
letters = re.compile("[A-Za-z]")
e_12 = re.compile("E\+12")

for i in range(len(gr)):
    if letters.search(gr.isbn13[i]) != None:
        if e_12.search(gr.isbn13[i]) == None:
            gr.loc[i, 'isbn13'] = 'None'

            
## Remove rows with missing (and non-valid) isbn numbers
gr = gr[gr.isbn13 != 'None']
gr.index = range(len(gr))


## Expand ISBN numbers from E+12 format
for i in range(len(gr)):
    gr.loc[i, 'isbn13'] = str(int(pd.to_numeric(gr.loc[i, 'isbn13'])))

---

#### Cleaning Goodreads IDs

The Goodreads IDs scraped from goodreads do not match up with the IDs stored in OL. This code will stay here for now as reference in case something changes and we do want to use this identifier. But otherwise it will currently be commented out.

In [9]:
### Goodreads IDs
#
### Pull out Goodreads ids from 'editions_url' column
#gr_id_patt = re.compile("(?<=/work/editions/)[0-9]+")
#
#id_vec = []
#
#for ed in gr.editions_url:
#    gr_id = gr_id_patt.search(ed)
#    if gr_id != None:
#        id_vec.append(gr_id[0])
#    else:
#        id_vec.append(None)
#        
#gr["gr_id"] = id_vec
#
### Drop missing gr_id rows (for now)
#gr.drop(gr.loc[gr["gr_id"].isnull()].index, inplace = True)

---

# Open Library python package

This can take individual ISBN values (or other identifiers) and return the associated json file.

Issues:
- It is likely slower than using the bulk file since it will have to re-query for each line.
- Any slowness is going to scale up as data increases

Good stuff:
- Some books have multiple ISBNs that are close in values. The bulk json files make it difficult to use these slighlty different ISBN numbers to pull data. The OL package immediately recognizes the similar ISBN values and pulls data accordingly.
- It can pull based on other identifiers *** To test: can we used the Amazon identifiers to get a few more books in there? If so, the above cleaning method will need to be updated to seperate out observations with Amazon IDs.


https://github.com/internetarchive/openlibrary-client/blob/master/olclient/openlibrary.py

In [2]:
from olclient.openlibrary import OpenLibrary

In [5]:
## Example from OL documentation
ol = OpenLibrary()
work = ol.Work.get(u'OL12938932W')
editions = work.editions

In [7]:
## Example format for pulling book info for particular ISBN
isbn_test = ol.Edition.get(isbn = '9781477823835')

In [14]:
## Take ISBN numbers from Goodreads dataset
isbn_list = gr.isbn13.unique()

In [18]:
keyvec = ["isbn_13", "title", "genres", "subjects", "description"]

ol_data = []

for isbn in isbn_list:
    book = ol.Edition.get(isbn = isbn)
    if book != None:
        book_dat = [book.json().get(key) for key in keyvec]
        ol_data.append(book_dat)

In [28]:
isbn_df = pd.DataFrame(ol_data, columns = keyvec)

,isbn_13,title,genres,subjects,description
0,[9780060838720],Bel canto,[Fiction.],"[Women singers -- Fiction., Victims of terrori...","{'type': '/type/text', 'value': 'Somewhere in ..."
1,[9781484705278],Every last word,None,"[Schools, High schools, Obsessive-compulsive d...","{'type': '/type/text', 'value': 'Consumed by a..."
2,[9780765379955],All the Birds in the Sky,None,None,"{'type': '/type/text', 'value': 'An ancient so..."
3,None,Cosmos,[Popular works.],[Astronomy -- Popular works.],None
4,None,MEDITATIONS; TRANS. BY MARTIN HAMMOND.,None,None,None
...,...,...,...,...,...
1866,[9780451459879],Blood rites,None,None,None
1867,[9780006479673],If Tomorrow Comes,None,None,None
1868,[9781600965272],Orthodoxy,None,"[Literary Collections / Essays, Religion / Cat...",None
1869,[9780380713806],Neither here nor there,None,"[Bryson, Bill -- Travel -- Europe., Europe -- ...",None


In [27]:
## Direct match rate
nomatch = len(isbn_list) - len(isbn_df)
total = len(isbn_list)

print("Unmatched ISBN count:", nomatch)

print("Unmatched ISBN proportion:", round(nomatch / total * 100, 2), "%")

Unmatched ISBN count: 107
Unmatched ISBN proportion: 5.41 %


In [43]:
## Subject/genre rate

val_vec = []

for i in range(len(isbn_df)):
    val = 0
    if isbn_df.genres[i] != None:
        val += 1
    if isbn_df.subjects[i] != None:
        val += 2
    if isbn_df.description[i] != None:
        val += 4

    val_vec.append(val)
    
val_vec = np.array(val_vec)

In [48]:
noinfo = ((val_vec == 0) | (val_vec == 4)).sum()
print("Number of ISBNs with no subject or genre info:", noinfo, "(", round((noinfo / total) * 100, 2), "%)")

print("Number of ISBNs with no info:", (val_vec == 0).sum())

Number of ISBNs with no subject or genre info: 884 ( 44.69 %)
Number of ISBNs with no info: 820


# Open Library Bulk Data

### Basic look at data formatting

Data can be found here:

https://openlibrary.org/developers/dumps

!wc -l ../data/edition_json.txt

!head -1 ../data/edition_json.txt

### ISBN Merge

In [160]:
## Take ISBN numbers from Goodreads dataset
isbn_list = gr.isbn13.to_list()
isbn_patt = re.compile("(?<=isbn_13\': \[\').{13}")

In [124]:
## Pull appropriate key values from json file
keys = ["isbn_13", "title", "genre", "subjects"]
ol_vec = []

with open('../data/edition_json.txt') as json_file:
    for line in json_file:
        if (re.search('\"isbn_13\"', line) != None): 
            if isbn_patt.search(line) != None:
                isbn_num = isbn_patt.search(line)[0]
                if str(isbn_num) in isbn_list:
                    l = [json.loads(line).get(key) for key in keys]
                    ol_vec.append(l)

In [125]:
## Convert open library list to dataframe with properly formatted ISBN numbers
isbn_df = pd.DataFrame(ol_vec, columns = keys)

clean_isbn = []

for l in isbn_df.isbn_13:
    clean_isbn.append(l[0])

isbn_df["isbn13"] = clean_isbn

In [126]:
merge_check = gr.merge(isbn_df, on = 'isbn13')
#merge_check

In [127]:
gr_merge = gr.merge(isbn_df, how = 'left', on = 'isbn13')
#gr_merge

In [128]:
gr_merge

,book_id,book_author,book_language,num_reviews,num_ratings,avg_rating,isbn13,editions_url,book_publication_date,book_first_publication_date,series,data_log_time,isbn_13,title,genre,subjects
0,5826,Ann Patchett,English,15165,239360,3.93,9780060838720,/work/editions/859342-bel-canto,August 2 2005,May 22 2001,None,10/22/2020 9:48,[9780060838720],Bel canto,None,"[Women singers -- Fiction., Victims of terrori..."
1,5826,Ann Patchett,English,15165,239360,3.93,9780060838720,/work/editions/859342-bel-canto,August 2 2005,May 22 2001,None,10/22/2020 9:48,[9780060838720],Bel Canto,None,None
2,5826,Ann Patchett,English,15165,239360,3.93,9780060838720,/work/editions/859342-bel-canto,August 2 2005,May 22 2001,None,10/22/2020 9:48,[9780060838720],Bel Canto (P.S.),None,"[Popular American Fiction, Fiction - General, ..."
3,23341894,Tamara Ireland Stone,English,4975,37318,4.25,9781484705278,/work/editions/26212678-every-last-word,June 16 2015,None,None,10/22/2020 9:49,[9781484705278],Every last word,None,"[Schools, High schools, Obsessive-compulsive d..."
4,25372801,Charlie Jane Anders,English,6111,37471,3.59,9780765379948,/work/editions/45119441-all-the-birds-in-the-sky,January 26 2016,None,All the Birds in the Sky,10/22/2020 9:49,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2436,27,Bill Bryson,English,2895,58989,3.86,9780380713806,/work/editions/3164459-neither-here-nor-there-...,March 28 1993,1991,Bryson and Katz,10/26/2020 10:41,[9780380713806],Neither here nor there,None,"[Bryson, Bill -- Travel -- Europe, Europe -- D..."
2437,23341607,Elin Hilderbrand,English,3691,50139,3.73,9780316334525,/work/editions/42901131-the-rumor,June 16 2015,None,None,10/26/2020 10:42,[9780316334525],The rumor,None,"[Gossip, Fiction, Women novelists, Female frie..."
2438,22522202,Abbi Glines,English,3720,39448,3.99,9781481438865,/work/editions/41969628-until-friday-night,August 25 2015,None,The Field Party,10/26/2020 10:43,NaN,NaN,NaN,NaN
2439,22521286,Charlie N. Holmberg,English,7039,67883,3.65,9781477823835,/work/editions/40051132-the-paper-magician,September 1 2014,None,The Paper Magician,10/26/2020 10:45,NaN,NaN,NaN,NaN


Merging on ISBN numbers covers about 50% of the dataset

##### Next steps:

- Try the goodreads ID numbers - refer below to the test.txt data to see how they appear in the OL data. Will need to pull them out of the GR data > take a string of numbers of any length proceeded by '/work/editions/'

- Make sure merge works with that identifier--should be the same as using ISBNs once cleaned

- Check what isn't merging -- are the books weird? Do those titles appear in OL but with a different identifier? Do they not appear in OL at all? Can they be pulled through the OL package rather than the bulk file?

### Goodreads ID Merge

In [78]:
## Take ISBN numbers from Goodreads dataset
gr_id_list = gr.gr_id.to_list()
gr_patt = re.compile("(?<=goodreads\": \[\")[0-9]+")

In [79]:
## Pull appropriate key values from json file
keys = ["identifiers", "isbn_13", "title", "genres", "subjects"]
ol_vec = []

with open('../data/edition_json.txt') as json_file:
    for line in json_file:
        if (re.search('\"goodreads\"', line) != None): 
            if gr_patt.search(line) != None:
                gr_num = gr_patt.search(line)[0]
                if gr_num in gr_id_list:
                    l = [json.loads(line).get(key) for key in keys]
                    ol_vec.append(l)

In [84]:
## Convert open library list to dataframe with properly formatted ISBN numbers
ol_df = pd.DataFrame(ol_vec, columns = keys)

In [85]:
gr_vec = []

for i in range(len(ol_df)):
    gr_vec.append(ol_df.identifiers[i]["goodreads"][0])
    
ol_df["gr_id"] = gr_vec
#ol_df.drop(columns = "identifiers", inplace = True)

In [87]:
merge_check = gr.merge(ol_df, on = 'gr_id')
#merge_check

In [90]:
gr_merge = gr.merge(ol_df, how = 'left', on = 'gr_id')
#gr_merge

## Scratch/checking

In [19]:
## Pull appropriate key values from json file
keys = ["identifiers", "isbn_13", "title", "subjects"]
ol = []

with open('../data/test.txt') as json_file:
    for line in json_file:
        if (re.search('\"goodreads\"', line) != None): 
            if gr_patt.search(line) != None:
                l = [json.loads(line).get(key) for key in keys]
                ol.append(l)

In [21]:
pd.DataFrame(ol)

,0,1,2,3
0,{'goodreads': ['6850240']},[9780107805401],"Parliamentary Debates, House of Lords, Bound V...","[Government - Comparative, Politics / Current ..."
1,{'goodreads': ['2862283']},[9780107805852],"Parliamentary Debates, House of Lords, 1997-98","[Bibliographies, catalogues, discographies, PO..."
2,{'goodreads': ['3140331']},None,Helping kids resolve conflicts without violence,"[Violence in children -- Prevention., Children..."


In [1]:
keys = ["isbn_13", "title", "subjects"]
dat = []

count = 0

with open('../data/test.txt') as json_file:
    for line in json_file:
        print(line)



{"publishers": ["Bernan Press"], "physical_format": "Hardcover", "subtitle": "9th November - 3rd December, 1992", "key": "/books/OL10000135M", "title": "Parliamentary Debates, House of Lords, Bound Volumes, 1992-93", "identifiers": {"goodreads": ["6850240"]}, "isbn_13": ["9780107805401"], "languages": [{"key": "/languages/eng"}], "number_of_pages": 64, "isbn_10": ["0107805405"], "publish_date": "December 1993", "last_modified": {"type": "/type/datetime", "value": "2010-04-24T17:54:01.503315"}, "authors": [{"key": "/authors/OL2645777A"}], "latest_revision": 4, "works": [{"key": "/works/OL7925046W"}], "type": {"key": "/type/edition"}, "subjects": ["Government - Comparative", "Politics / Current Events"], "revision": 4}

{"publishers": ["Stationery Office"], "physical_format": "Hardcover", "subtitle": "26 January - 4 February 1998", "title": "Parliamentary Debates, House of Lords, 1997-98", "isbn_10": ["0107805855"], "identifiers": {"goodreads": ["2862283"]}, "isbn_13": ["9780107805852"],

In [125]:
keys = ["isbn_13", "title", "subjects"]
dat = []

with open('../data/test.txt') as json_file:
    for line in json_file:
        l = [json.loads(line).get(key) for key in keys]
        dat.append(l)

dat

[[None,
  'colonização italiano no Vale do Itajaí-Mirim',
  ['Italians -- Brazil -- Itajaí-Mirim River Valley',
   'Italians -- Brazil',
   'Itajaí-Mirim River Valley (Brazil) -- History']]]

In [155]:
data = []
with open('../data/test.txt') as f:
    for line in f:
        data.append(json.loads(line))

In [156]:
data

[{'publishers': ['Bernan Press'],
  'physical_format': 'Hardcover',
  'subtitle': '9th November - 3rd December, 1992',
  'key': '/books/OL10000135M',
  'title': 'Parliamentary Debates, House of Lords, Bound Volumes, 1992-93',
  'identifiers': {'goodreads': ['6850240']},
  'isbn_13': ['9780107805401'],
  'languages': [{'key': '/languages/eng'}],
  'number_of_pages': 64,
  'isbn_10': ['0107805405'],
  'publish_date': 'December 1993',
  'last_modified': {'type': '/type/datetime',
   'value': '2010-04-24T17:54:01.503315'},
  'authors': [{'key': '/authors/OL2645777A'}],
  'latest_revision': 4,
  'works': [{'key': '/works/OL7925046W'}],
  'type': {'key': '/type/edition'},
  'subjects': ['Government - Comparative', 'Politics / Current Events'],
  'revision': 4},
 {'publishers': ['Stationery Office'],
  'physical_format': 'Hardcover',
  'subtitle': '26 January - 4 February 1998',
  'title': 'Parliamentary Debates, House of Lords, 1997-98',
  'isbn_10': ['0107805855'],
  'identifiers': {'goodre

In [154]:
keys = ["isbn_13", "title", "subjects", "source_records"]
dat = []

with open('../data/test.txt') as json_file:
    for line in json_file:
        if re.search(re.compile('subject'), line) != None:
            l = [json.loads(line).get(key) for key in keys]
            dat.append(l)

dat

[[['9780107805401'],
  'Parliamentary Debates, House of Lords, Bound Volumes, 1992-93',
  ['Government - Comparative', 'Politics / Current Events'],
  None],
 [['9780107805852'],
  'Parliamentary Debates, House of Lords, 1997-98',
  ['Bibliographies, catalogues, discographies',
   'POLITICS & GOVERNMENT',
   'Reference works',
   'Bibliographies & Indexes',
   'Reference'],
  None],
 [['9780108360374'],
  'Local Government (Contracts) Bill (House of Lords Bills)',
  ['English law: contract law'],
  None],
 [None,
  'Helping kids resolve conflicts without violence',
  ['Violence in children -- Prevention.',
   'Children and violence.',
   'Conflict management.'],
  None],
 [['9780108380280'],
  'Scottish Enterprise Bill (House of Lords Bills)',
  ['Laws of Other Jurisdictions & General Law'],
  None]]